In [1]:
import os
import time
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import (
    CharacterTextSplitter,
)
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from colorama import Fore
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGUAGE_MODEL = "gpt-3.5-turbo-instruct"


In [ ]:


template: str = """/
    تو یک مفسر مثنوی مولانا هستی. سعی کن جواب سوال : /
      {question} /
      :رو بر اساس منبع روبرو /
      {context}
    بدهی. /
    """

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

model = ChatOpenAI()


In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])



def load_documents():
    """Load all text files from a directory, split them into chunks, and return the split documents."""
    loader = DirectoryLoader("./output_text/", glob="*.txt")  # Load all .txt files
    raw_documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )
    return text_splitter.split_documents(raw_documents)



def load_embeddings(documents, user_query):
    """Create or load a vector store from a set of documents."""
    persist_directory = './chroma_cache'  # Directory to store embeddings
    embedding_model = OpenAIEmbeddings()

    # Check if the persist directory exists
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory)
        # Initialize Chroma and add documents
        db = Chroma.from_documents(documents, embedding_model, persist_directory=persist_directory)
        db.persist()
    else:
        # Load existing Chroma instance
        db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

    docs = db.similarity_search(user_query, k=5)
    print("\nRetrieved Documents:\n")
    print(format_docs(docs))  # Print the retrieved documents
    return db.as_retriever()


def generate_response(retriever, query):
    pass
    # Create a prompt template using a template from the config module and input variables
    # representing the context and question.
    # create the prompt

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | chat_prompt_template
        | model
        | StrOutputParser()
    )
    return chain.invoke(query)

def query(query):
    documents = load_documents()
    retriever = load_embeddings(documents, query)
    response = generate_response(retriever, query)
    return response

In [ ]:
documents = load_documents()

In [18]:
retriever = load_embeddings(documents, """تفسیر بیت زیر چیست: /"
"یاد من کن پیش تخت آن عزیز /
تا مرا هم واخرد زین حبس نیز""")


Retrieved Documents:

دگر باره بشوریدن، بدانسانه، بجان تو، که هر بندی که بر بندی، بدرانن.به جان تو چوچرخم من، چوماهم من چوماهم من، چوشمم منچوشم من زتاب تو، زتاب توهم عقلم، هم عشقم، هم جانمبه جان توآیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیایی آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آیای آی آیای آیای آی آی آی آی آی آی آی آی آینخواهم عمر فانی را...تویی عمر عزیز مننخواهم جان پر قیم را بادتویی جانم به جان تواگر بیتو بر افلام کرد تو ابرتی میر

In [19]:
response = generate_response(retriever, """تفسیر بیت زیر چیست: /"
"یاد من کن پیش تخت آن عزیز /
تا مرا هم واخرد زین حبس نیز""")

print(response)

این بیت از مثنوی مولانا به معنای "یاد من کن و پیش تخت آن عزیز بیاور / تا هم من از این زندانی نیز رهایی یابم" است. در این بیت، شاعر به عزیز خود اشاره کرده و از او خواسته است که به یادش بیاورد و در کمک او باشد تا از دست زندانی شدن خود نیز رهایی یابد. این بیت می‌تواند به معنای تلاش برای آزادی از بندگی و محدودیت‌ها، به‌ویژه در مسیر روحانیت و عشق الهی باشد.
